# zib gereed maken voor mapping

* inlezen en layout consistent maken met XtEHR
* opslaan voor mapping - zie mapping Notebook

## imports

In [7]:
import pandas as pd
import os

## toon de zibs in folder (hier: zib2024)

In [8]:
base = 'zibs2024'
folder = os.path.join(base, 'downloads')
files = os.listdir(folder)
[(i, file) for i, file in enumerate(files)]

[(0, 'nl.zorg.AdministrationAgreement-v3.0(2024EN).xlsx'),
 (1, 'Nl.zorg.Alert-v5.0(2024EN).xlsx'),
 (2, 'Nl.zorg.Condition-v1.1(2024EN).xlsx'),
 (3, 'Nl.zorg.Diagnosis-v2.0(2024EN).xlsx'),
 (4, 'nl.zorg.DispenseRequest-v4.0(2024EN).xlsx'),
 (5, 'Nl.zorg.HealthcareProvider-v3.6(2024EN).xlsx'),
 (6, 'Nl.zorg.HealthProfessional-v4.0(2024EN).xlsx'),
 (7, 'Nl.zorg.LaboratoryTestResult-v7.1(2024EN).xlsx'),
 (8, 'Nl.zorg.MedicalDevice-v5.0(2024EN).xlsx'),
 (9, 'Nl.zorg.NursingIntervention-v5.0(2024EN).xlsx'),
 (10, 'Nl.zorg.part.AddressInformation-v1.2(2024EN).xlsx'),
 (11, 'Nl.zorg.part.AnatomicalLocation-v1.0.4(2024EN).xlsx'),
 (12, 'Nl.zorg.part.ContactInformation-v1.3.1(2024EN).xlsx'),
 (13, 'Nl.zorg.part.InstructionsForUse-v3.1(2024EN).xlsx'),
 (14, 'Nl.zorg.part.NameInformation-v1.2(2024EN).xlsx'),
 (15, 'Nl.zorg.Patient-v4.3(2024EN).xlsx'),
 (16, 'Nl.zorg.Procedure-v6.0(2024EN).xlsx'),
 (17, 'Nl.zorg.process.OrderData-v1.0.1(2024EN).xlsx'),
 (18, 'Nl.zorg.process.RegistrationData-v1.1

## kies een zib

In [9]:
file = files[17]
print(file)
url = os.path.join(folder, file)
zibdf = pd.read_excel(url, sheet_name='Data', header=1)
zibdf

Nl.zorg.process.OrderData-v1.0.1(2024EN).xlsx


C:\Users\zanen_nict1\AppData\Roaming\Python\Python313\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Unnamed: 0,Concept,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Alias,Type,Card.,Stereotype,Id,Definition,DefinitionCode,Reference,Constraints
0,NaN,OrderData,NaN,NaN,NaN,NaN,NaN,NL: AanvraagGegevens,NaN,NaN,rootconcept,NL-CM:22.2.1,Root concept of the OrderData process data inf...,NaN,NaN,NaN
1,NaN,NaN,OrderDateTime,NaN,NaN,NaN,NaN,NL: AanvraagDatumTijd,TS,1,data,NL-CM:22.2.2,Date and if relevant time when the order is made.,NaN,NaN,NaN
2,NaN,NaN,OrderId,NaN,NaN,NaN,NaN,NL: Aanvraagnummer,II,1,data,NL-CM:22.2.3,Order identification number.,NaN,tbd,NaN
3,NaN,NaN,Requester::HealthProfessional,NaN,NaN,NaN,NaN,NL: Aanvrager::Zorgverlener,NaN,1,"data,reference",NL-CM:22.2.4,"The person who ordered the action (procedure, ...",NaN,This is a reference to the rootconcept of info...,NaN
4,NaN,NaN,OrderReason,NaN,NaN,NaN,NaN,NL: RedenAanvraag,ST,0..1,data,NL-CM:22.2.5,"Context of the order, which is relevant for th...",NaN,NaN,NaN
5,NaN,NaN,ClinicalQuestion,NaN,NaN,NaN,NaN,NL: Vraagstelling,ST,0..1,data,NL-CM:22.2.6,Description of the question to which the reque...,NaN,NaN,NaN
6,NaN,NaN,Status,NaN,NaN,NaN,NaN,NL: Status,CD,1,data,NL-CM:22.2.7,"Status of the order in the ordering process, s...",NaN,StatusCodelist,NaN
7,NaN,NaN,Comment,NaN,NaN,NaN,NaN,NL: Opmerking,ST,0..1,data,NL-CM:22.2.8,Additional information that is important for t...,LOINC: 48767-8 Annotation comment [Interpretat...,NaN,NaN


## opschonen zib

* lowercase kolomnamen
* paar labels laten vervallen
* name en path kolom toevoegen
* name vullen (naam attribuut)
* kolommen opschonen tot benodigde

Handmatig:
* path vullen (als in logical model, dus pad.pad2.pad3.naam etc.)

In [10]:
zibdf.columns = [col.lower() for col in zibdf.columns]
zibdf = zibdf.drop(labels=['unnamed: 0', 'reference'], axis=1).dropna(how='all')
zibdf.insert(0, 'name', None)
zibdf.insert(0, 'path', None)
zibdf.fillna('')
concepts = []
for i, row in zibdf.iterrows():
    # Loop thru columns 2 tot 8 (Concept and 'Unnamed' ones)
    for j in range(2, 6):
        # print(j, row.iloc[j])
        if not pd.isna(row.iloc[j]):
            # Remove last element if next column is empty
            if len(concepts) > j -2:
                concepts.pop()
            try:
                concepts[j - 2] = row.iloc[j]
            except IndexError:
                concepts.append(row.iloc[j])
    # print(i, concepts)
    zibdf.at[i, 'path'] = '.'.join(concepts)
    zibdf.at[i, 'name'] = concepts[-1]
zibdf = zibdf[['name', 'path', 'alias', 'type', 'card.', 'stereotype', 'id', 'definition', 'definitioncode']].fillna('')
zibdf

,name,path,alias,type,card.,stereotype,id,definition,definitioncode
0,OrderData,OrderData,NL: AanvraagGegevens,,,rootconcept,NL-CM:22.2.1,Root concept of the OrderData process data inf...,
1,OrderDateTime,OrderData.OrderDateTime,NL: AanvraagDatumTijd,TS,1,data,NL-CM:22.2.2,Date and if relevant time when the order is made.,
2,OrderId,OrderData.OrderId,NL: Aanvraagnummer,II,1,data,NL-CM:22.2.3,Order identification number.,
3,Requester::HealthProfessional,OrderData.Requester::HealthProfessional,NL: Aanvrager::Zorgverlener,,1,"data,reference",NL-CM:22.2.4,"The person who ordered the action (procedure, ...",
4,OrderReason,OrderData.OrderReason,NL: RedenAanvraag,ST,0..1,data,NL-CM:22.2.5,"Context of the order, which is relevant for th...",
5,ClinicalQuestion,OrderData.ClinicalQuestion,NL: Vraagstelling,ST,0..1,data,NL-CM:22.2.6,Description of the question to which the reque...,
6,Status,OrderData.Status,NL: Status,CD,1,data,NL-CM:22.2.7,"Status of the order in the ordering process, s...",
7,Comment,OrderData.Comment,NL: Opmerking,ST,0..1,data,NL-CM:22.2.8,Additional information that is important for t...,LOINC: 48767-8 Annotation comment [Interpretat...


## Excel opslaan

In [11]:
url = os.path.join(base, 'mappable', file)
zibdf.to_excel(url, index=False)